In [1]:
pip install --upgrade pip setuptools wheel

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyproject-toml

In [3]:
pip install gensim

  error: subprocess-exited-with-error
  
  × Building wheel for gensim (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [714 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\gensim
      copying gensim\downloader.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\interfaces.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\matutils.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\nosy.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\utils.py -> build\lib.win-amd64-cpython-312\gensim
      copying gensim\__init__.py -> build\lib.win-amd64-cpython-312\gensim
      creating build\lib.win-amd64-cpython-312\gensim\corpora
      copying gensim\corpora\bleicorpus.py -> build\lib.win-amd64-cpython-312\gensim\corpora
      copying gensim\corpora\csvcorpus.py -> bui


  Using cached gensim-4.3.2.tar.gz (23.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build gensim


In [4]:
import os.path
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

ModuleNotFoundError: No module named 'gensim'

In [ ]:
def carregar_dados(path,nome_arq):
    lista_documentos = []
    titulos=[]
    with open( os.path.join(path, nome_arq) ,"r") as fin:
        for line in fin.readlines():
            text = line.strip()
            lista_documentos.append(text)
    titulos.append( text[0:min(len(text),100)] )
    return lista_documentos,titulos

In [ ]:
def processar_dados(doc_set):
    # inicializar o tokenizer regex
    tokenizer = RegexpTokenizer(r'\w+')
    # cria uma lista de stopwords em inglês
    stopword_ing = set(stopwords.words('english'))
    # Crie p_stemmer da classe PorterStemmer
    p_stemmer = PorterStemmer()
    # lista de documentos tokenizados
    texts = []
    # percorrer a lista de documentos
    for i in doc_set:
        # limpar e tokenizar a string do documento
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words dos tokens
        stopped_tokens = [i for i in tokens if not i in stopword_ing]
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # adiciona tokens a lista
        texts.append(stemmed_tokens)
    return texts

In [ ]:
def preparar_corpus(documento_preparado):
    # Criar o dicionário de termos do nosso courpus, onde cada termo único é atribuído a um índice.
    dictionary = corpora.Dictionary(documento_preparado)
    dictionary = corpora.Dictionary(documento_preparado)
    # Convertendo a lista de documentos (corpus) em Document Term Matrix usando o dicionário preparado acima.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in documento_preparado]
    # geramodelo LDA
    return dictionary,doc_term_matrix

In [ ]:
def cria_modelo_LSA(documento_preparado,qtd_topicos,words):
    dictionary,doc_term_matrix=preparar_corpus(documento_preparado)
    # gera o modelo LSA
    lsamodel = LsiModel(doc_term_matrix, num_topics=qtd_topicos, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=qtd_topicos, num_words=words))
    return lsamodel

In [ ]:
def calculo_coerencia(dictionary, doc_term_matrix, documento_preparado, stop, start=2, step=3):
    valor_coerencia = []
    model_list = []
    for qtd_topicos in range(start, stop, step):
        # gera modelo LSA
        model = LsiModel(doc_term_matrix, num_topics=qtd_topicos, id2word = dictionary)  
        # modelo de treinamento
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=documento_preparado, dictionary=dictionary, coherence='c_v')
        valor_coerencia.append(coherencemodel.get_coherence())
    return model_list, valor_coerencia

In [ ]:
def plot_graph(documento_preparado,start, stop, step):
    dictionary,doc_term_matrix=preparar_corpus(documento_preparado)
    model_list, valor_coerencia = calculo_coerencia(dictionary, doc_term_matrix,documento_preparado,
                                                            stop, start, step)
    # mostra o gráfico
    x = range(start, stop, step)
    plt.plot(x, valor_coerencia)
    plt.xlabel("Número de tópicos")
    plt.ylabel("Score de coerência")
    plt.legend(("valor_coerencia"), loc='best')
    plt.show()

In [ ]:
qtdd_topicos=4
words=10
lista_documentos,titulos=carregar_dados("","../Processamento_linguagem_natural/data/noticias.txt")
limpar_texto=processar_dados(lista_documentos)
model=cria_modelo_LSA(limpar_texto,qtdd_topicos,words)

start,stop,step=2,4,1
plot_graph(limpar_texto,start,stop,step)